<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/9_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rule 1

單一點落在3外

#### 1.1 一般 (未分群，只查看 value)

In [ ]:
import pandas as pd

def check_western_electric_rule1(df, column_name, k):
    """
    檢查 Western Electric Rule 1 是否被觸發。

    參數:
    df (pandas.DataFrame): 包含過程數據的 DataFrame。
    column_name (str): DataFrame 中包含過程變數的列名。
    k (float): 控制限的倍數，通常為 3 倍標準差。

    返回:
    violations (pandas.DataFrame): 包含違規點的 DataFrame。
    """
    # 計算平均值和標準差
    mean = df[column_name].mean()
    std_dev = df[column_name].std()

    # 計算控制限
    upper_control_limit = mean + k * std_dev
    lower_control_limit = mean - k * std_dev

    # 檢測違規的點
    mask = (df[column_name] > upper_control_limit) | (df[column_name] < lower_control_limit)
    violations = df[mask]

    return violations

# 示例使用
# 假設 'process_data' 是一個 DataFrame，並且我們關心名為 'measurement' 的列
# process_data = pd.read_excel('path_to_your_file.xlsx')  # 加載你的數據
# rule1_violations = check_western_electric_rule1(process_data, 'measurement', 3)
# print(rule1_violations)


#### 1.2 有分群

## Rule 2

連續9點同一側

In [ ]:
import pandas as pd

def western_electric_rule2(df, k, cal_col, mean, filter_ls=None):
    record = []

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - k + 1):
            sub_df = df[cal_col].iloc[i:i+k]
            # 檢查是否連續 k 點都大於均值或小於均值
            if all(sub_df > mean) or all(sub_df < mean):
                # 如果找到符合條件的連續 k 點，記錄這些點的第 k 點（即第9點）
                record.append(df.iloc[i + k - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - k + 1):
                sub_df = group[cal_col].iloc[i:i+k]
                # 檢查是否連續 k 點都大於均值或小於均值
                if all(sub_df > mean) or all(sub_df < mean):
                    # 如果找到符合條件的連續 k 點，記錄這些點的第 k 點（即第9點）
                    record.append(group.iloc[i + k - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


## Rule 3 (已做過)

In [ ]:
import pandas as pd

def western_electric_rule3(df, k, cal_col, filter_ls=None):
    record = []

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - k + 1):
            sub_df = df[cal_col].iloc[i:i+k]
            # 檢查是否單調遞增或單調遞減
            if all(sub_df.diff().dropna() > 0) or all(sub_df.diff().dropna() < 0):
                # 如果找到符合條件的連續 k 點，記錄這些點的第 k 點
                record.append(df.iloc[i + k - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - k + 1):
                sub_df = group[cal_col].iloc[i:i+k]
                # 檢查是否單調遞增或單調遞減
                if all(sub_df.diff().dropna() > 0) or all(sub_df.diff().dropna() < 0):
                    # 如果找到符合條件的連續 k 點，記錄這些點的第 k 點
                    record.append(group.iloc[i + k - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


## Rule 4

In [ ]:
import pandas as pd

def western_electric_rule4(df, k, cal_col, filter_ls=None):
    record = []

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - k + 1):
            sub_df = df[cal_col].iloc[i:i+k].values
            # 檢查是否交替上升下降
            if all((sub_df[j] < sub_df[j+1] and sub_df[j+1] > sub_df[j+2]) or
                   (sub_df[j] > sub_df[j+1] and sub_df[j+1] < sub_df[j+2])
                   for j in range(k - 2)):
                # 如果找到符合條件的連續 k 點，記錄這些點的第 k 點
                record.append(df.iloc[i + k - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - k + 1):
                sub_df = group[cal_col].iloc[i:i+k].values
                # 檢查是否交替上升下降
                if all((sub_df[j] < sub_df[j+1] and sub_df[j+1] > sub_df[j+2]) or
                       (sub_df[j] > sub_df[j+1] and sub_df[j+1] < sub_df[j+2])
                       for j in range(k - 2)):
                    # 如果找到符合條件的連續 k 點，記錄這些點的第 k 點
                    record.append(group.iloc[i + k - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


說明：
1. 使用 all()：在主程式中直接使用 all() 函數來檢查 sub_df 是否符合交替上升或下降的條件。條件是：

  * sub_df[j] < sub_df[j+1] and sub_df[j+1] > sub_df[j+2]：檢查第 j 和 j+1 點是上升，然後 j+1 和 j+2 是下降。
  * sub_df[j] > sub_df[j+1] and sub_df[j+1] < sub_df[j+2]：檢查第 j 和 j+1 點是下降，然後 j+1 和 j+2 是上升。
2. 遍歷的長度：我們只需要遍歷到 k-2，因為每次檢查三個相鄰的點。

3. 分群條件：如果有 filter_ls，會先對 df 進行分群，然後對每個分群中的資料進行相同的檢查。

In [ ]:
def western_electric_rule4_v2(df, k, col_name):
    # 初始化一個空的 DataFrame 來存儲結果
    columns = ['col1', 'col2', 'col_value']
    for i in range(k, 0, -1):
        columns.append(f'point{i}')
        columns.append(f'point{i}_value')

    results = pd.DataFrame(columns=columns)

    # 檢查每個點是否符合連續 k 點交替上升下降的規則
    for i in range(len(df) - k + 1):
        sub_df = df[col_name].iloc[i:i+k].values
        # 檢查是否交替上升下降
        is_alternating = all((sub_df[j] > sub_df[j-1] and sub_df[j+1] < sub_df[j]) or
                             (sub_df[j] < sub_df[j-1] and sub_df[j+1] > sub_df[j])
                             for j in range(1, k-1))
        if is_alternating:
            # 如果找到符合條件的連續 k 點，記錄這些點及其索引
            row = {
                'col1': df.iloc[i+k-1]['col1'],  # 根據實際需求自行調整
                'col2': df.iloc[i+k-1]['col2'],  # 根據實際需求自行調整
                'col_value': df[col_name].iloc[i+k-1]
            }
            for j in range(k):
                row[f'point{k-j}'] = i+k-j
                row[f'point{k-j}_value'] = df[col_name].iloc[i+k-j]

            results = results.append(row, ignore_index=True)

    return results

# 測試函數
result_rule4_v2 = western_electric_rule4_v2(df, 14, 'col_name')
tools.display_dataframe_to_user(name="Western Electric Rule 4 Results (Corrected)", dataframe=result_rule4_v2)


## Rule 5

連續三點中的兩點落在 2 sigma之外 (與中線距離超過 2 sigma)

(好像是之前2/3 ooc)

In [ ]:
import pandas as pd

def western_electric_rule_custom(df, cal_col, mean, sigma, filter_ls=None):
    record = []

    # 計算 2 sigma 的上下界限
    upper_limit = mean + 2 * sigma
    lower_limit = mean - 2 * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - 2):
            sub_df = df[cal_col].iloc[i:i+3].values
            # 檢查三個點中的兩個點是否落在 2 sigma 之外
            outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
            if sum(outside_sigma) >= 2:
                # 如果找到符合條件的情況，記錄這些點的第 3 點
                record.append(df.iloc[i + 2])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - 2):
                sub_df = group[cal_col].iloc[i:i+3].values
                # 檢查三個點中的兩個點是否落在 2 sigma 之外
                outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
                if sum(outside_sigma) >= 2:
                    # 如果找到符合條件的情況，記錄這些點的第 3 點
                    record.append(group.iloc[i + 2])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


* 改版 (確保最後一個點是)

In [ ]:
import pandas as pd

def western_electric_custom_rule(df, k, cal_col, mean, sigma, filter_ls=None):
    record = []

    # 定義 2 sigma 的上下界限
    upper_limit = mean + 2 * sigma
    lower_limit = mean - 2 * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - k + 1):
            sub_df = df[cal_col].iloc[i:i+k].values
            # 計算每一個點是否落在 2 sigma 外
            is_outside_sigma = [(val > upper_limit or val < lower_limit) for val in sub_df]

            # 檢查連續三點中的至少兩點落在 2 sigma 之外，且最後一個點必須符合
            if sum(is_outside_sigma) >= 2 and is_outside_sigma[-1]:
                # 如果找到符合條件的連續 k 點，記錄這些點的最後一點
                record.append(df.iloc[i + k - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - k + 1):
                sub_df = group[cal_col].iloc[i:i+k].values
                # 計算每一個點是否落在 2 sigma 外
                is_outside_sigma = [(val > upper_limit or val < lower_limit) for val in sub_df]

                # 檢查連續三點中的至少兩點落在 2 sigma 之外，且最後一個點必須符合
                if sum(is_outside_sigma) >= 2 and is_outside_sigma[-1]:
                    # 如果找到符合條件的連續 k 點，記錄這些點的最後一點
                    record.append(group.iloc[i + k - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


**函數參數說明：**

* df: 輸入的 DataFrame。
* cal_col: 要檢查的數值欄位名稱。
* mean: 中心線的均值。
* sigma: 數據的標準差。
* n1: 連續幾個點需要檢查（例如：3點）。
* n2: 需要超過標準差範圍的點數（例如：2點）。
* k: 幾個標準差的倍數（例如：2倍標準差）。
* filter_ls: 分群條件，可以為空、1個或2個，用於 groupby。


---



**主要邏輯：**
1. 計算 k 倍標準差的上下限，即 mean + k * sigma 和 mean - k * sigma。
2. 檢查每 n1 個連續點中是否有 n2 點超出這個範圍。
3. 使用 sum(outside_sigma) 計算超過上下限的點數，若滿足條件（超過 n2 點），則記錄第 n1 點。

In [ ]:

import pandas as pd

def western_electric_rule_general(df, cal_col, mean, sigma, n1, n2, k, filter_ls=None):
    record = []

    # 計算 k 個標準差的上下界限
    upper_limit = mean + k * sigma
    lower_limit = mean - k * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - n1 + 1):
            sub_df = df[cal_col].iloc[i:i+n1].values
            # 檢查 n1 點中有多少點落在 k sigma 之外
            outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
            if sum(outside_sigma) >= n2:
                # 如果找到符合條件的情況，記錄這些點的第 n1 點
                record.append(df.iloc[i + n1 - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - n1 + 1):
                sub_df = group[cal_col].iloc[i:i+n1].values
                # 檢查 n1 點中有多少點落在 k sigma 之外
                outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
                if sum(outside_sigma) >= n2:
                    # 如果找到符合條件的情況，記錄這些點的第 n1 點
                    record.append(group.iloc[i + n1 - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


* 改版 (最後一個點要是)

In [ ]:
import pandas as pd

def western_electric_custom_rule(df, n1, n2, cal_col, mean, sigma, k_sigma, filter_ls=None):
    record = []

    # 定義 k sigma 的上下界限
    upper_limit = mean + k_sigma * sigma
    lower_limit = mean - k_sigma * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - n1 + 1):
            sub_df = df[cal_col].iloc[i:i+n1].values
            # 計算每一個點是否落在 k sigma 外
            is_outside_sigma = [(val > upper_limit or val < lower_limit) for val in sub_df]

            # 檢查連續 n1 點中至少有 n2 點落在 k sigma 之外，且最後一個點必須符合
            if sum(is_outside_sigma) >= n2 and is_outside_sigma[-1]:
                # 如果找到符合條件的連續 n1 點，記錄這些點的最後一點
                record.append(df.iloc[i + n1 - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - n1 + 1):
                sub_df = group[cal_col].iloc[i:i+n1].values
                # 計算每一個點是否落在 k sigma 外
                is_outside_sigma = [(val > upper_limit or val < lower_limit) for val in sub_df]

                # 檢查連續 n1 點中至少有 n2 點落在 k sigma 之外，且最後一個點必須符合
                if sum(is_outside_sigma) >= n2 and is_outside_sigma[-1]:
                    # 如果找到符合條件的連續 n1 點，記錄這些點的最後一點
                    record.append(group.iloc[i + n1 - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


## Rule 6  (同rule5)

## Rule 7

連續十五點落在 1 sigma之內 (與中線距離少於 1 sigma)

In [ ]:
import pandas as pd

def western_electric_rule_sigma(df, cal_col, mean, sigma, n1, k, filter_ls=None):
    record = []

    # 計算 1 sigma 的上下界限
    upper_limit = mean + k * sigma
    lower_limit = mean - k * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - n1 + 1):
            sub_df = df[cal_col].iloc[i:i+n1].values
            # 檢查 n1 點是否都落在 1 sigma 之內
            inside_sigma = [(x < upper_limit and x > lower_limit) for x in sub_df]
            if all(inside_sigma):
                # 如果找到符合條件的情況，記錄這些點的第 n1 點
                record.append(df.iloc[i + n1 - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - n1 + 1):
                sub_df = group[cal_col].iloc[i:i+n1].values
                # 檢查 n1 點是否都落在 1 sigma 之內
                inside_sigma = [(x < upper_limit and x > lower_limit) for x in sub_df]
                if all(inside_sigma):
                    # 如果找到符合條件的情況，記錄這些點的第 n1 點
                    record.append(group.iloc[i + n1 - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


## Rule 8

連續八點都在 1 sigma 之外的上下兩側 (與中線距離超過 1 sigma)

In [ ]:
import pandas as pd

def western_electric_rule_sigma_outside(df, cal_col, mean, sigma, n1, k, filter_ls=None):
    record = []

    # 計算 1 sigma 的上下界限
    upper_limit = mean + k * sigma
    lower_limit = mean - k * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - n1 + 1):
            sub_df = df[cal_col].iloc[i:i+n1].values
            # 檢查 n1 點是否都落在 1 sigma 之外
            outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
            if all(outside_sigma):
                # 如果找到符合條件的情況，記錄這些點的第 n1 點
                record.append(df.iloc[i + n1 - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - n1 + 1):
                sub_df = group[cal_col].iloc[i:i+n1].values
                # 檢查 n1 點是否都落在 1 sigma 之外
                outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
                if all(outside_sigma):
                    # 如果找到符合條件的情況，記錄這些點的第 n1 點
                    record.append(group.iloc[i + n1 - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)


* 檢查「連續八點都在 1 sigma 之外的上下兩側」，且要求這八點中**至少有一點**在上方超過 1 sigma，另一點在下方低於 1 sigma

In [ ]:
import pandas as pd

def western_electric_rule_two_sides(df, cal_col, mean, sigma, n1, k, filter_ls=None):
    record = []

    # 計算 1 sigma 的上下界限
    upper_limit = mean + k * sigma
    lower_limit = mean - k * sigma

    # 如果 filter_ls 為空，直接對整個 DataFrame 進行檢查
    if not filter_ls:
        for i in range(len(df) - n1 + 1):
            sub_df = df[cal_col].iloc[i:i+n1].values
            # 檢查 n1 點是否都落在 1 sigma 之外
            outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
            # 確保至少一點在上方(> upper_limit)，至少一點在下方(< lower_limit)
            has_upper = any(x > upper_limit for x in sub_df)
            has_lower = any(x < lower_limit for x in sub_df)

            # 如果都在 1 sigma 之外且兩側都有點
            if all(outside_sigma) and has_upper and has_lower:
                # 如果找到符合條件的情況，記錄這些點的第 n1 點
                record.append(df.iloc[i + n1 - 1])

    # 如果有分群條件
    else:
        grouped = df.groupby(filter_ls)
        for name, group in grouped:
            for i in range(len(group) - n1 + 1):
                sub_df = group[cal_col].iloc[i:i+n1].values
                # 檢查 n1 點是否都落在 1 sigma 之外
                outside_sigma = [(x > upper_limit or x < lower_limit) for x in sub_df]
                # 確保至少一點在上方(> upper_limit)，至少一點在下方(< lower_limit)
                has_upper = any(x > upper_limit for x in sub_df)
                has_lower = any(x < lower_limit for x in sub_df)

                # 如果都在 1 sigma 之外且兩側都有點
                if all(outside_sigma) and has_upper and has_lower:
                    # 如果找到符合條件的情況，記錄這些點的第 n1 點
                    record.append(group.iloc[i + n1 - 1])

    # 回傳符合條件的紀錄
    return pd.DataFrame(record)
